sewing machine, refrigerator, typewriter, Bookcase, Frame, lumber, TV, Audio, fax machine, chair, dish dryer, bicycle, arcade machine, TV stand, audio cabinet, Wardrobe, Display cabinet, Cupboard, Rice storage container, Stroller, Signboard, Washing machine, Speaker, Desk, Vanity table, Wall clock, Earthenware jar, Water tank, Air conditioner, Fan, Dining table, Small cabinet, Mannequin, Sink, Bedding items, Gas oven range, Heater, Cabinet, Drawer unit, Clothes drying rack, Toilet bowl, Spin dryer, Electric rice cooker, Kitchen sink, Shoe rack, Trunk, suitcase, Bathtub, Air purifier, Video player, Bed, Coat rack, Display stand, Floor covering
Microwave: Vacuum cleaner, Table, Sofa, Door panel, Mat ,Bamboo mat, Computer ,Main unit, Monitor, Printer, Humidifier, Bookstand, Piano, Aquarium, Carpet, PP bag, General waste bag

In [ ]:
import onnxruntime as ort
import numpy as np
import cv2
import os
# ONNX 모델 로드
ort_session = ort.InferenceSession("yolow-l.onnx")

# 입력 이미지 준비
image_path = 'KakaoTalk_Photo_2024-10-22-10-01-15.jpeg'
image = cv2.imread(image_path)
original_height, original_width = image.shape[:2]

# 모델 입력 크기에 맞게 이미지 조정
input_image = cv2.resize(image, (640, 640))  # YOLO-World 입력 크기에 맞게 조정
input_image = input_image.transpose(2, 0, 1)  # 채널 순서 변경 (HWC -> CHW)
input_image = np.expand_dims(input_image, axis=0)  # 배치 차원 추가
input_image = input_image.astype(np.float32) / 255.0  # 정규화

# 입력 이름 확인
input_name = ort_session.get_inputs()[0].name

# 모델 실행
outputs = ort_session.run(None, {input_name: input_image})

# 출력 해석
boxes = outputs[1][0]  # 바운딩 박스 좌표 (640x640 이미지 기준)
scores = outputs[2][0]  # 신뢰도
class_ids = outputs[3][0]  # 클래스 ID
confidence_threshold = 0.1

# 다수의 객체를 처리하는 반복문
for i in range(len(scores)):
    score = scores[i]
    if score > confidence_threshold:  # 신뢰도가 임계값보다 높은 객체만 처리
        x1, y1, x2, y2 = boxes[i]  # 바운딩 박스 좌표 (640x640 기준)
        
        # 바운딩 박스 좌표를 원본 이미지 크기에 맞게 변환
        x1 = int(x1 / 640 * original_width)
        y1 = int(y1 / 640 * original_height)
        x2 = int(x2 / 640 * original_width)
        y2 = int(y2 / 640 * original_height)
        
        class_id = int(class_ids[i])  # 클래스 ID
        label = f"Class {class_id} {score:.2f}"

        # 색상 지정
        color = (0, 255, 0)  # 초록색
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)  # 바운딩 박스 그리기
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# 결과 이미지 표시
cv2.imshow('Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 검출된 객체 정보 출력
print("검출된 객체 정보:")
for i in range(len(scores)):
    score = scores[i]
    if score > confidence_threshold:
        class_id = int(class_ids[i])
        print(f"클래스 ID: {class_id}, 신뢰도: {score:.2f}")


In [ ]:
import onnxruntime as ort
import numpy as np
import cv2
import os

# ONNX 모델 로드
ort_session = ort.InferenceSession("yolow-l.onnx")

# 이미지가 저장된 폴더 경로
folder_path = 'image'

# 폴더 내 모든 파일 반복
for filename in os.listdir(folder_path):
    # 파일의 확장자가 이미지 형식인지 확인
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
        image_path = os.path.join(folder_path, filename)  # 전체 이미지 경로

        # 입력 이미지 준비
        image = cv2.imread(image_path)
        original_height, original_width = image.shape[:2]

        # 모델 입력 크기에 맞게 이미지 조정
        input_image = cv2.resize(image, (640, 640))  # YOLO-World 입력 크기에 맞게 조정
        input_image = input_image.transpose(2, 0, 1)  # 채널 순서 변경 (HWC -> CHW)
        input_image = np.expand_dims(input_image, axis=0)  # 배치 차원 추가
        input_image = input_image.astype(np.float32) / 255.0  # 정규화

        # 입력 이름 확인
        input_name = ort_session.get_inputs()[0].name

        # 모델 실행
        outputs = ort_session.run(None, {input_name: input_image})

        # 출력 해석
        boxes = outputs[1][0]  # 바운딩 박스 좌표 (640x640 이미지 기준)
        scores = outputs[2][0]  # 신뢰도
        class_ids = outputs[3][0]  # 클래스 ID
        confidence_threshold = 0.1

        # 다수의 객체를 처리하는 반복문
        for i in range(len(scores)):
            score = scores[i]
            if score > confidence_threshold:  # 신뢰도가 임계값보다 높은 객체만 처리
                x1, y1, x2, y2 = boxes[i]  # 바운딩 박스 좌표 (640x640 기준)

                # 바운딩 박스 좌표를 원본 이미지 크기에 맞게 변환
                x1 = int(x1 / 640 * original_width)
                y1 = int(y1 / 640 * original_height)
                x2 = int(x2 / 640 * original_width)
                y2 = int(y2 / 640 * original_height)

                class_id = int(class_ids[i])  # 클래스 ID
                label = f"Class {class_id} {score:.2f}"

                # 색상 지정
                color = (0, 255, 0)  # 초록색
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)  # 바운딩 박스 그리기
                cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # 결과 이미지 표시
        cv2.imshow('Result', image)
        cv2.waitKey(0)  # 결과 창에서 키 입력을 기다림
        cv2.destroyAllWindows()

        # 검출된 객체 정보 출력
        print(f"파일: {filename}")
        print("검출된 객체 정보:")
        for i in range(len(scores)):
            score = scores[i]
            if score > confidence_threshold:
                class_id = int(class_ids[i])
                print(f"클래스 ID: {class_id}, 신뢰도: {score:.2f}")

